In [ ]:
import torch
torch.set_default_dtype(torch.float64)
import tdg

import tdg.HMC as HMC
from tqdm.notebook import tqdm

import tdg.plot as visualize
import matplotlib.pyplot as plt

import h5py as h5

Let us try to compare against some data in [2212.05177](https://arxiv.org/abs/2212.05177).

First, to establish their notation:

$$\begin{align}
    \alpha(x) &= -\left(\log x a\right)^{-1} 
    &
    \alpha    &= \alpha(k_F) \text{ if no argument is provided.}
\end{align}$$

We should aim for $\alpha>0$ so that we do not need to compute the gap $M \Delta^2/4\pi$ (5.1).  Their results are good up to $\alpha \lesssim 0.6$ or so.

What does that correspond to in terms of the number density
$$ \rho = N/L^2 = \frac{g}{4\pi} k_F^2 ?$$

Massaging the definition of $\alpha$ one finds
$$\begin{align}
    e^{-1/\alpha} = k_F a = \sqrt{\frac{4\pi N}{gL^2}} a = \sqrt{\frac{N}{g\pi}} \frac{2\pi a}{L}
\end{align}$$

Since for our system $g=2$,
$$\begin{align}
    e^{-1/\alpha} = \sqrt{\frac{N}{2\pi}} \frac{2\pi a}{L} = \sqrt{\frac{N}{2\pi}} \tilde{a}
\end{align}$$

Suppose we want to stay in the dilute regime $\text{sparsity} = N/\Lambda\lesssim 0.1 $ to keep spatial discretization errors at bay.  Then we should solve
$$\begin{align}
    e^{-1/\alpha} = \sqrt{\frac{\text{sparsity} \Lambda}{2\pi}} \tilde{a}
\end{align}$$
for $\tilde{a}$ to find
$$\begin{align}
    \tilde{a} = \frac{e^{-1/\alpha}}{\sqrt{\frac{\Lambda \text{sparsity}}{2\pi}}}
\end{align}$$

In [ ]:
def target_ere(alpha, lattice, sparsity):
    
    atilde = torch.exp(-1/alpha) / torch.sqrt( lattice.sites * sparsity / (2*torch.pi))
    
    return tdg.EffectiveRangeExpansion(torch.tensor((atilde,)))

They show the contact density [their (7.5)] for a range of α in Fig. 12.  Something to note is that the energy density in their definition [their (5.1)] requires subtracting off the gap.

We can access the attractive channel with α < 0.  

In [ ]:
alpha_target = torch.tensor(-0.3)
Lattice = tdg.Lattice(7)
sparsity = torch.tensor(0.1) # so we should tune µ until we find about 5 particles
Z = tdg.Luescher.Zeta2D()

We can tune to the continuum limit.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))

# First draw the zeta function.
exact = torch.linspace(-5.001, 30.001, 1000)
Z.plot(ax, exact, color='gray')

# and the analytic piece of the ERE
ere = target_ere(alpha_target, Lattice, sparsity)
analytic = ere.analytic(exact)
ax.plot(exact, analytic.clone().detach(), color='black', linestyle='dashed')

def plot(ax, nx, alpha, starting_lattice, starting_sparsity):
    Lattice = tdg.Lattice(nx)
    # Take the continuum limit holding the number of particles fixed
    # and increasing the number of sites.
    sparsity = starting_sparsity*(starting_lattice.nx/nx)**2
    
    ere = target_ere(alpha, Lattice, sparsity)
    tuning = tdg.AnalyticTuning(ere, Lattice)
    A1 = tdg.ReducedTwoBodyA1Hamiltonian(Lattice, [tdg.LegoSphere(r) for r in tuning.radii])
    E  = A1.eigenenergies(tuning.C)
    x  = E * Lattice.sites / (2*torch.pi)**2
    z  = Z(x) / torch.pi**2
    ax.plot(
        x.clone().detach().numpy(),
        z.clone().detach().numpy(),
        linestyle='none', marker='o',
        label=f'nx={Lattice.nx}'
        )

for nx in range(7,25,2):
    plot(ax, nx, alpha_target, Lattice, sparsity)

ax.legend()
ax.set_xlim((min(exact), max(exact)))
ax.set_ylim((-0,5))

Now that we know the Hamiltonian would converge to the right thing, let us prepare a many-body calculation!

In [ ]:
tuning = tdg.AnalyticTuning(ere, Lattice)
nt=32
beta = 3
mu = 0. # Iterated over a few choices until N came out about right.  Is it a coincidence that it's 0?

In [ ]:
S = tuning.Action(nt, beta, torch.tensor(mu))
print(S)

In [ ]:
H = HMC.Hamiltonian(S)
integrator = HMC.Omelyan(H, 30, 1)
hmc = HMC.MarkovChain(H, integrator)

In [ ]:
configurations = 1000 # takes about 20 minutes on my laptop

# Read from a stored ensemble if possible.
try:
    with h5.File('./many-body-contact-comparison.h5', 'r') as f:
        ensemble = tdg.ensemble.GrandCanonical.from_h5(f['/ensemble'])
        if len(ensemble.configurations) < configurations:
            raise
        else:
            ensemble.configurations = ensemble.configurations[-configurations:]
except:
    ensemble = tdg.ensemble.GrandCanonical(S).generate(configurations, hmc, start='hot', progress=tqdm)
    with h5.File('./many-body-contact-comparison.h5', 'w') as f:
        ensemble.to_h5(f.create_group('/ensemble'))

In [ ]:
viz = visualize.History(2)
viz.plot(ensemble.N('fermionic').real, 0, label=f'N = {ensemble.N("fermionic").mean().real:.4f}')
viz.plot(ensemble.S.real, 1)
viz.ax[0,0].legend()

Now we can evaluate $k_F a = \sqrt{N/\pi g} \tilde{a}$ and check where we got α close to our target value, given the β and μ we picked.

In [ ]:
kFa = torch.sqrt(ensemble.N('fermionic').mean() / (2*torch.pi)) * ere.a
print(f'{kFa=:.4}')
alpha = -1/torch.log(kFa)
print(f"α  :{alpha_target:.4f} target")
print(f"α  ={alpha.real:.4} measured")

Great!  A close value!  Let's look at the contact!

In [ ]:
viz = visualize.History()
viz.plot(ensemble.contact('fermionic').real, 0, label=f'contact = {ensemble.contact("fermionic").mean().real:.4f}')
viz.ax[0,0].legend()

Beane et al. report $C/k_F^4$ in their Figure 12.

Their $C$ is the contact *density*.  

To make a fair comparison we should compute $\Lambda (C \Delta x^2) / (2 \pi N)^2$.

In [ ]:
def C_by_kF4(ensemble):
    return ensemble.Action.Spacetime.Lattice.sites * ensemble.contact('fermionic').mean() / (2*torch.pi * ensemble.N('fermionic').mean())**2

In [ ]:
print(f"C/kF^4 = {C_by_kF4(ensemble).real:.4f}")

Which is compatible with the curve in Figure 12.  Note that we haven't subtracted off the gap squared, so this agreement may be spurious?  Or, perhaps, because we're at finite temperature we might not be in the superfluid phase and the gap need not be subtracted?  Perhaps the resolution is for both signs of α in (5.1) the energy density is given by the Hamiltonian?